In [1]:
import pdm_functions as fns
import torch 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
# df_normal = fns.csv_to_tensor_stack('dataset/5528_spec_normal.csv')
# df_error = fns.csv_to_tensor_stack('dataset/5528_spec_error.csv')

In [3]:
# num_rows = df_error.shape[0]
# grouped_tensor_error = df_error.view(num_rows, 12, 6144)

In [4]:
# grouped_tensor_error.shape

In [5]:
# def get_random_permutation(num_groups):
#     seed = torch.randint(0, 10000, (1,)).item()  # Generate a random seed
#     torch.manual_seed(seed)  # Set the random seed
#     return torch.randperm(num_groups)

# # Example usage
# for _ in range(5):  # Generate 5 different permutations
#     permuted_indices = get_random_permutation(12)
#     print(permuted_indices)

In [6]:
# permuted_tensors_list_error = []

# for _ in range(20): 
#     permuted_indices = fns.get_random_permutation(12)
#     permuted_tensor = grouped_tensor_error[:, permuted_indices, :]
    
#     permuted_tensors_list_error.append(permuted_tensor)    

In [7]:
# permuted_tensors_list_error[0].shape

In [8]:
# stacked_tensor_error = torch.cat(permuted_tensors_list_error, dim=0)
# stacked_tensor_error.shape

In [9]:
# final_tensor_error = stacked_tensor_error.view(-1, 73728)
# final_tensor_error.shape

In [10]:
# def multi_datasets(df, multi_dim=2): 
#     permuted_tensors_list = []
#     num_rows = df.shape[0]
#     grouped_tensor = df.view(num_rows, 12, 6144)

#     for _ in range(20): 
#         permuted_indices = fns.get_random_permutation(12)
#         permuted_tensor = grouped_tensor[:, permuted_indices, :]
        
#         permuted_tensors_list_.append(permuted_tensor)

#     stacked_tensor = torch.cat(permuted_tensors_list, dim=0)
#     final_tensor = stacked_tensor.view(-1, 73728)

#     return final_tensor
    

In [11]:
# bogus_error = fns.multi_datasets(df_error, multi_dim= 4)

In [12]:
f_normal = pd.read_csv('dataset/5528_drop_imbalance_normal.csv')

In [13]:
f_error = pd.read_csv('dataset/5528_drop_imbalance_error.csv')

In [14]:
# f_normal.head()

In [15]:
# created_at 유닉스 시간대 컬럼을 기준으로 시계열 정렬
f_normal['created_at'] = pd.to_datetime(f_normal['created_at'], unit='s')
f_normal = f_normal.sort_values(by='created_at')
 
f_error['created_at'] = pd.to_datetime(f_error['created_at'], unit='s')
f_error = f_error.sort_values(by='created_at')

In [16]:
f_normal = f_normal.drop(columns=['asset_id', 'created_at', 'created_at_datetime', 'looseness_health', 'time','misalignment_health', 'bearing_health', 'imbalance_health'])
f_error = f_error.drop(columns=['asset_id', 'created_at', 'created_at_datetime', 'looseness_health', 'time','misalignment_health', 'bearing_health', 'imbalance_health'])



In [17]:
f_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2220 entries, 0 to 2219
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   rms_x               2220 non-null   float64
 1   rms_y               2220 non-null   float64
 2   rms_z               2220 non-null   float64
 3   rms_xyz             2220 non-null   float64
 4   vel_rms_x           2220 non-null   float64
 5   vel_rms_y           2220 non-null   float64
 6   vel_rms_z           2220 non-null   float64
 7   vel_rms_xyz         2220 non-null   float64
 8   skewness_x          2220 non-null   float64
 9   skewness_y          2220 non-null   float64
 10  skewness_z          2220 non-null   float64
 11  vel_skewness_x      2220 non-null   float64
 12  vel_skewness_y      2220 non-null   float64
 13  vel_skewness_z      2220 non-null   float64
 14  kurtosis_x          2220 non-null   float64
 15  kurtosis_y          2220 non-null   float64
 16  kurtos

In [38]:
import pdm_functions as fns 
import torch

In [39]:
f_normal_tensor = torch.tensor(f_normal.values, dtype=torch.float32)
f_error_tensor = torch.tensor(f_error.values, dtype=torch.float32)


In [20]:
f_normal_tensor.shape, f_error_tensor.shape

(torch.Size([2220, 38]), torch.Size([192, 38]))

In [21]:
f_normal_test = f_normal_tensor[:1200]
f_normal_train = f_normal_tensor[1200:]

In [22]:
f_normal_train.shape, f_normal_test.shape

(torch.Size([1020, 38]), torch.Size([1200, 38]))

In [23]:
f_normal_test = torch.concat((f_normal_test, f_error_tensor), dim= 0)

In [24]:
f_normal_test.shape

torch.Size([1392, 38])

In [25]:
f_normal_test = f_normal_test.reshape((-1, 1, 12, 38))
f_normal_test.shape

torch.Size([116, 1, 12, 38])

In [32]:
f_normal_train = f_normal_train.reshape((-1, 1, 12, 38))
f_normal_train.shape

torch.Size([85, 1, 12, 38])

In [27]:
asdf

NameError: name 'asdf' is not defined

In [33]:
f_normal_train_aug = fns.multi_datasets(f_normal_train, multi_dim= 150, num_groups= 12)

100%|███████████████████████████████████████████████████████████████████████████████| 150/150 [00:00<00:00, 487.02it/s]


In [34]:
f_normal_train_aug.shape

torch.Size([5814000, 1])

In [35]:
f_normal_train_aug = f_normal_train_aug.reshape((-1, 1, 12, 38))
f_normal_train_aug.shape

torch.Size([12750, 1, 12, 38])

In [31]:
f_test = f_normal_test
f_test.shape

torch.Size([116, 1, 12, 38])

In [37]:
torch.save({'train_normal' : f_normal_train_aug, 'test' : f_test }, 'final_experiment/datasets/auto_encoder.pt')

In [ ]:
f_normal = fns.multi_datasets(f_normal, multi_dim= 10, num_groups= 12)

In [ ]:
# tensors = [torch.tensor(item) for item in f_normal.groupby(level=0).apply(lambda x: [item for sublist in x for item in sublist])]
    
#     # # 텐서들을 스택
    # result = torch.stack(tensors)

In [ ]:
f_normal_tensor = torch.tensor(f_normal.values, dtype=torch.float32)
f_error_tensor = torch.tensor(f_error.values, dtype=torch.float32)


In [ ]:
f_normal_tensor.shape, f_error_tensor.shape

In [ ]:
data_auto = torch.concat((f_normal_tensor, f_error_tensor), dim=0)

In [ ]:
data_auto

In [ ]:
data_auto_tensor = data_auto.reshape((-1, 1, 12, 38))

In [ ]:
# tensor_data = tensor_data.reshape((-1 ,12 ,38))
data_auto_tensor.shape

In [ ]:
data_auto = fns.multi_datasets_stacks(data_auto_tensor, multi_dim = 500, num_groups= 12)


In [ ]:
data_auto.shape

In [ ]:
torch.save(data_auto, 'dataset/train_datasets/sigdata_aug.pt')